### k_fold.ipynb
- Alessandro Trincone
- Mario Gabriele Carofano

> ...

In [ ]:
import auxfunc
import constants
from artificial_neural_network import NeuralNetwork
from training_report import TrainingReport
from training_params import TrainingParams
import dataset_functions as df
import plot_functions as pf

import numpy as np
import pprint
import time
from datetime import datetime
import copy
import gc

import pandas as pd
import random
import os

...

In [ ]:
def k_fold_cross_validation(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
        l_sizes : list[int] = constants.DEFAULT_HIDDEN_LAYER_NEURONS,
        params : TrainingParams = None
) -> tuple[float, float, float, float]:
    
    """
        E' una tecnica di validazione che utilizza una parte indipendente del training set per la fase di validazione. Si utilizza per la selezione degli iper-parametri che restituiscono il minor errore di validazione sull'addestramento del modello.

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.
        -   l_sizes : e' una lista contenente le dimensioni di uno o piu' hidden layer e dell'output layer della rete neurale.
        -   params : e' un oggetto della classe TrainingParams che contiene alcuni iperparametri per la fase di addestramento della rete neurale.

        Returns:
        -   err_mean : la media dei valori di errore di validazione su tutti i modelli addestrati.
        -   err_std : la deviazione standard dei valori di errore di validazione su tutti i modelli addestrati.
        -   acc_mean : la media delle percentuali di accuracy di validazione su tutti i modelli addestrati.
        -   acc_std : la deviazione standard delle percentuali di accuracy di validazione su tutti i modelli addestrati.
    """

    fold_reports = []

    Xfolds, Yfolds = df.split_dataset(Xtrain, Ytrain, k)

    for i in range(k):

        print(f"\nFold {i+1} di {k}")

        # Tutti gli altri parametri di NeuralNetwork sono inizializzati con i valori default
        net = NeuralNetwork(l_sizes=l_sizes)

        training_fold = np.concatenate([fold for j, fold in enumerate(Xfolds) if j != i])
        training_labels = np.concatenate([fold for j, fold in enumerate(Yfolds) if j != i])
        validation_fold = Xfolds[i]
        validation_labels = Yfolds[i]

        history_report = net.train(
            training_fold, training_labels,
            validation_fold, validation_labels,
            params
        )

        accs = [r.validation_accuracy for r in history_report]
        errs = [r.validation_error for r in history_report]

        fold_reports.append({
            "Fold"      : i+1,
            "Report"    : copy.deepcopy(net.training_report),
            "E_mean"    : np.mean(errs),
            "E_std"     : np.std(errs),
            "E_min"     : np.min(errs),
            "E_max"     : np.max(errs),
            "A_mean"    : np.mean(accs),
            "A_std"     : np.std(accs),
            "A_min"     : np.min(accs),
            "A_max"     : np.max(accs)
        })

        del net
        del training_fold, training_labels
        del validation_fold, validation_labels
        gc.collect()

        if constants.DEBUG_MODE:
            break

    # end for i

    # Disegno di un line plot con le percentuali di accuracy, media e deviazione standard su tutte le fold.
    acc_mean, acc_std = pf.plot_k_fold_accuracy_scores(out_directory, fold_reports)

    # Disegno di un line plot con i valori di errore, media e deviazione standard su tutte le fold.
    err_mean, err_std = pf.plot_k_fold_error_scores(out_directory, fold_reports)

    return err_mean, err_std, acc_mean, acc_std

# end

...

In [ ]:
def grid_search_cv(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
):
    
    """
        ...

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.

        Returns:
        -   ... : ...
    """

    # Il valore tipico per "eta minus" e' compreso tra 0.5 e 0.9.
    # Valori più vicini a 0.5 riducono il passo di aggiornamento in modo più aggressivo, migliorando la stabilita' ma potenzialmente rallentando la convergenza.
    # Valori più vicini a 0.9 sono meno aggressivi, permettendo una convergenza più rapida ma con il rischio di instabilita'.
    eta_minus_values = [0.5, 0.7, 0.9]

    # Il valore tipico per "eta plus" e' compreso tra 1.2 e 1.5.
    # Valori più vicini a 1.2 incrementano il passo di aggiornamento in modo più moderato, migliorando la stabilita' e riducendo il rischio di oscillazioni.
    # Valori più vicini a 1.5 incrementano il passo di aggiornamento in modo più aggressivo, accelerando la convergenza ma aumentando il rischio di instabilita'.
    eta_plus_values = [1.2, 1.3, 1.5]

    # Per problemi di classificazione (es. riconoscimento di cifre scritte a mano), un intervallo comune per il numero di neuroni è da 10 a 100 per uno o due hidden layer.
    # Tuttavia, siccome l'input ha alta dimensionalita', potrebbe essere necessario aumentare il numero di neuroni per catturare le caratteristiche importanti. Ad esempio, per immagini 28x28, l'hidden layer puo' avere un numero di neuroni compreso tra 128 e 512.
    hidden_layer_values = [128, 256, 512]

    # Si inizializza un "DataFrame" per la raccolta delle metriche di valutazione.
    stats = pd.DataFrame({
        "Eta minus"     : [],
        "Eta plus"      : [],
        "Hidden layer"  : [],
        "Mean error"    : [],
        "Std error"     : [],
        "Mean accuracy" : [],
        "Std accuracy"  : []
    })

    # Si calcolano tutte le possibili combinazioni di iper-parametri.
    combs = [
        (x, y, z)
        for x in eta_minus_values
        for y in eta_plus_values
        for z in hidden_layer_values
    ]

    # Questo ciclo "for" esamina tutte le possibili combinazioni di iper-parametri.
    for i, (em, ep, hl) in enumerate(combs):

        print(f"Combinazione {i+1}:\n\tEta minus = {em},\n\tEta plus = {ep},\n\tHidden layer = {hl}\n")

        # Si applica la tecnica di validazione "K-fold cross validation" per valutare la combinazione di iper-parametri corrente. 
        # err_mean, err_std, acc_mean, acc_std = k_fold_cross_validation(
        #     out_directory,
        #     Xtrain, Ytrain, k,
        #     [hl, 10],
        #     params=TrainingParams(
        #         epochs=2,
        #         eta_minus=em,
        #         eta_plus=ep
        #     )
        # )
        err_mean, err_std, acc_mean, acc_std = (0,0,0,0)

        # Si crea un dizionario per raccogliere tutte le metriche di valutazione riguardo la rete appena addestrata sulla combinazione di iperparametri corrente.
        new_row = {
            "Eta minus"     : em,
            "Eta plus"      : ep,
            "Hidden layer"  : int(hl),
            "Mean error"    : err_mean,
            "Std error"     : err_std,
            "Mean accuracy" : acc_mean,
            "Std accuracy"  : acc_std
        }

        # Si aggiunge il dizionario come nuova riga alla tabella generale.
        stats = pd.concat([stats, pd.DataFrame([new_row])], ignore_index=True)

        # Si ripristina l'indice per mantenere l'ordinamento cronologico.
        stats = stats.reset_index(drop=True)
    
    # end for i, (em, ep, hl)

    # Si visualizzano le statistiche raccolte in una tabella.
    display(stats)

    # Infine, si salva la tabella in un file ".csv".
    os.makedirs(out_directory, exist_ok=True)
    stats.to_csv(out_directory + 'stats.csv', index=False)

# end

# RIFERIMENTI
# https://saturncloud.io/blog/how-to-insert-a-row-to-pandas-dataframe/
# https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/
# https://stackoverflow.com/questions/75956209/error-dataframe-object-has-no-attribute-append
# https://stackoverflow.com/questions/17006641/single-line-nested-for-loops

...

In [ ]:
def random_search_cv(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
        r : int = constants.DEFAULT_RANDOM_COMBINATIONS
):
    
    """
        ...

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.
        -   r : ...

        Returns:
        -   ... : ...
    """

    eta_minus_values = [0.5, 0.7, 0.9]
    eta_plus_values = [1.2, 1.3, 1.5]
    hidden_layer_values = [128, 256, 512]

    stats = pd.DataFrame({
        "Eta minus"     : [],
        "Eta plus"      : [],
        "Hidden layer"  : [],
        "Mean error"    : [],
        "Std error"     : [],
        "Mean accuracy" : [],
        "Std accuracy"  : []
    })

    combs = [
        (x, y, z)
        for x in eta_minus_values
        for y in eta_plus_values
        for z in hidden_layer_values
    ]

    # Si utilizza il metodo "shuffle" per riordinare in modo casuale le combinazioni di iper-parametri.
    random.shuffle(combs)

    # Questo ciclo "for" esamina solo le prime "r" combinazioni di iper-parametri.
    for i in range(r):

        em = combs[i][0]
        ep = combs[i][1]
        hl = combs[i][2]
        
        print(f"Combinazione {i+1}:\n\tEta minus = {em},\n\tEta plus = {ep},\n\tHidden layer = {hl}\n")

        err_mean, err_std, acc_mean, acc_std = k_fold_cross_validation(
            out_directory,
            Xtrain, Ytrain, k,
            [hl, 10],
            params=TrainingParams(
                epochs=2,
                eta_minus=em,
                eta_plus=ep
            )
        )

        new_row = {
            "Eta minus"     : em,
            "Eta plus"      : ep,
            "Hidden layer"  : int(hl),
            "Mean error"    : err_mean,
            "Std error"     : err_std,
            "Mean accuracy" : acc_mean,
            "Std accuracy"  : acc_std
        }

        stats = pd.concat([stats, pd.DataFrame([new_row])], ignore_index=True)
        stats = stats.reset_index(drop=True)

    # end for i

    display(stats)

    os.makedirs(out_directory, exist_ok=True)
    stats.to_csv(out_directory + 'stats.csv', index=False)

# end

...

In [ ]:
idTrain, Xtrain, Ytrain, idTest, Xtest, Ytest = df.loadDataset(constants.COPPIE_TRAINING, constants.COPPIE_TEST)

...

In [ ]:
out_directory = constants.OUTPUT_DIRECTORY + datetime.now().strftime(constants.OUTPUT_DATE_TIME_FORMAT) + "/"
print(f"\nK-fold cross-validation iniziato: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
start_time = time.time()

grid_search_cv(out_directory, Xtrain, Ytrain)
random_search_cv(out_directory, Xtrain, Ytrain)

end_time = time.time()
tot_time = end_time - start_time

print(f"\nK-fold cross-validation completato: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
print(f"Tempo trascorso: {tot_time:.3f} secondi")

Se si vuole utilizzare una rete già addestrata con i seguenti parametri, iper-parametri e metriche di valutazione: <br>
NeuralNetwork( <br>
&emsp;depth = 2, <br>
&emsp;input_size = 784, <br>
&emsp;network_layers = [<br>
&emsp;Layer( <br>
&emsp;&emsp;size = 64, <br>
&emsp;&emsp;act_fun = <function leaky_relu at 0x10791c4c0>, <br>
&emsp;&emsp;inputs_size = (12500, 784) <br>
&emsp;&emsp;weights_shape = (64, 784), <br>
&emsp;&emsp;biases_shape = (64, 1) <br>
&emsp;), <br>
&emsp;Layer( <br>
&emsp;&emsp;size = 10, <br>
&emsp;&emsp;act_fun = <function identity at 0x107dda680>, <br>
&emsp;&emsp;inputs_size = (12500, 64) <br>
&emsp;&emsp;weights_shape = (10, 64), <br>
&emsp;&emsp;biases_shape = (10, 1) <br>
&emsp;)], <br>
&emsp;err_fun = <function cross_entropy_softmax at 0x107dda8c0>, <br>
&emsp;training_report = TrainingReport( <br>
&emsp;&emsp;num_epochs = 100, <br>
&emsp;&emsp;elapsed_time = 184.053 secondi, <br>
&emsp;&emsp;training_error = 1.83840, <br>
&emsp;&emsp;training_accuracy = 79.22 %, <br>
&emsp;&emsp;validation_error = 0.00000 <br>
&emsp;&emsp;validation_accuracy = 0.00 % <br>
&emsp;) <br>
)

In [ ]:
# net = NeuralNetwork.load_network_from_file("../output/2024-06-13_17-31/params.pkl")
# net = NeuralNetwork(
#     784, [64, 10],
#     l_act_funs=[auxfunc.leaky_relu, auxfunc.identity],
#     e_fun=auxfunc.cross_entropy_softmax
# )

# history_report = net.train(Xtrain, Ytrain, examples=len(Xtrain), rprop=True)
# pf.plot_error(out_directory, "rprop", [r.training_error for r in history_report])
# pf.plot_accuracy(out_directory, "rprop", [r.training_accuracy for r in history_report])

# print(repr(net))

...

In [ ]:
# net.test(
#     out_directory,
#     idTest, Xtest, Ytest,
#     plot_mode=constants.PlotTestingMode.ALL
# )

# net.predict(idTest, Xtest)